In [1]:
from lib.utils import load_raw_by_path
from torch import nn
from torch.nn.functional import batch_norm,relu
from torch import nn
import numpy as np
from torch.nn.functional import softmax
import torch
import sqlite3
from sqlite3 import Error
import os
import pandas as pd
from tqdm import tqdm
from lib.models import ResNet as MODEL
device = 'cuda'
EEG_1 = [1,8,14,15,16]
EEG_2 = [3,4,5,6,7,9,10,11,12,13,17]

ModuleNotFoundError: No module named 'lib'

In [2]:
model = torch.load('model.pt')
i = 1
raw = load_raw_by_path(f'data/aging/22-AGING-{i}.edf').get_data(picks=['EEG','EMG'])
if(i in EEG_1):
    eeg = raw[0]
elif(i in EEG_2):
    eeg = raw[1]
else:
    print("error")

In [3]:
X = torch.from_numpy(eeg.reshape(-1,5000))
del eeg
# center, stretch
X = (X - X.mean(axis=1,keepdim=True))/X.std(axis=1,keepdim=True)
# drop row if any element is inf
not_inf_idx = torch.where(~X.isinf().any(axis=1))[0]
X = X[not_inf_idx].float()

In [5]:
from torch.utils.data import DataLoader,TensorDataset
dataloader = DataLoader(TensorDataset(X),batch_size=64)
y_pred = torch.Tensor().cuda()
model.eval()
for (X_test) in dataloader:
    X_test = X_test[0].to(device)
    logits = model(X_test)
    y_pred = torch.cat([y_pred,torch.softmax(logits,dim=1).argmax(axis=1)])

/home/andrew/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,


In [16]:
pred_expert = np.array(y_pred.detach().cpu()).copy()
del y_pred
print("Experting")
for j in range(len(pred_expert)-2):
    if((pred_expert[j:j+2]==np.array([2,0])).all()):
        pred_expert[j+1] = 2
for j in range(len(pred_expert)-2):
    if(pred_expert[j+1] != pred_expert[j] and pred_expert[j+1] != pred_expert[j+2]):
        pred_expert[j+1] = pred_expert[j]
df = pd.DataFrame(pred_expert)

Experting


In [ ]:


dists = []
for i in tqdm(range(1,2)):
    if i==2:
        continue
    raw = load_raw_by_path(f'data/aging/22-AGING-{i}.edf').get_data(picks=['EEG','EMG'])
    if(i in EEG_1):
        eeg = raw[0]
    elif(i in EEG_2):
        eeg = raw[1]
    else:
        print("error")
    del raw

    y_pred = softmax(model(X.cuda()),dim=1).argmax(axis=1)
    del X
    pred_expert = np.array(y_pred.detach().cpu()).copy()
    del y_pred
    print("Experting")
    for j in range(len(pred_expert)-2):
        if((pred_expert[j:j+2]==np.array([2,0])).all()):
            pred_expert[j+1] = 2
    for j in range(len(pred_expert)-2):
        if(pred_expert[j+1] != pred_expert[j] and pred_expert[j+1] != pred_expert[j+2]):
            pred_expert[j+1] = pred_expert[j]
    df = pd.DataFrame(pred_expert)
    df[df[0] == 0] = 'P'
    df[df[0] == 1] = 'S'
    df[df[0] == 2] = 'W'
    print(df[0].value_counts(normalize=True))
    dists.append(df[0].value_counts(normalize=True))
    df.to_csv(f'data/aging/{i}_pred.csv',index=False)

    offset = 10e7       #epoch time period
    rename_dict = {'W':'Sleep-Wake', 'S':'Sleep-SWS', 'P':'Sleep-Paradoxical', 'X':''}

    csv_filename = f'data/aging/{i}_pred.csv'
    zdb_filename = f'data/aging/22-AGING-{i}.zdb'

    df = pd.read_csv(csv_filename)
    try:
        conn = sqlite3.connect(zdb_filename)
    except Error as e:
        print(e)
    cur = conn.cursor()

    #drop this table - creates issues
    query = "DROP TABLE IF EXISTS temporary_scoring_marker;"
    cur.execute(query)

    #get keyid of scoring
    query = "SELECT MAX(id) FROM scoring_revision WHERE name='Machine Data'"
    cur.execute(query)
    keyid = cur.fetchall()[0][0]

    #get starting point for scoring
    query = "SELECT id FROM scoring_marker WHERE type LIKE 'Sleep%' AND key_id='"+str(keyid)+"';"
    cur.execute(query)
    startid = cur.fetchall()[0][0]

    #get start time to crreate epochs
    query = 'SELECT starts_at FROM scoring_marker WHERE id = '+str(startid)+";"
    cur.execute(query)
    start_time = cur.fetchall()[0][0]
    stop_time = 0

    #delete first score before adding machine data
    query = "DELETE FROM scoring_marker;"
    cur.execute(query)

    #insert new epochs with scoring into the table
    for i in range(len(df)):
        #calculate epoch
        if i != 0:
            start_time = stop_time
        stop_time = start_time+offset

        score = rename_dict[df.at[i,'0']]
        #insert epoch
        query = f"""
                INSERT INTO scoring_marker 
                (starts_at, ends_at, notes, type, location, is_deleted, key_id)
                VALUES 
                ({start_time}, {stop_time}, '', '{score}', '', 0, {keyid});
                """ 
        cur.execute(query)

    conn.commit()
    conn.close()

In [ ]:
tmp = pd.DataFrame()
for dist in dists:
    tmp = pd.concat([tmp,dist],axis=1)

In [ ]:
import seaborn as sns
sns.kdeplot(tmp.T['P'].reset_index(drop=True),fill=True)
plt.xlabel("Proportion")
plt.title("Paradoxical Proportion Distribution Over Aging Cohort")
plt.savefig("p.jpg")

In [ ]:

sns.kdeplot(tmp.T['S'].reset_index(drop=True),color='green',fill=True)
plt.xlabel("Proportion")
plt.title("Slow Wave Proportion Distribution Over Aging Cohort")
plt.savefig("s.jpg")

In [ ]:

sns.kdeplot(tmp.T['W'].reset_index(drop=True),color='red',fill=True)
plt.xlabel("Proportion")
plt.title("Wakefulness Proportion Distribution Over Aging Cohort")
plt.savefig("w.jpg")